In [2]:
import tensorflow as tf
import tf_slim as slim

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import gym
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
env = gym.make('CartPole-v0')

In [16]:
gamma = 0.99

def discount_rewards(r): #Type Hinting 써볼까?
    discounted_rate = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_rate[t] = running_add
    return discounted_rate

class Agent:
    def __init__(self, lr, s_size, a_size, h_size):  #state, action
        self.state_in = tf.placeholder(shape=[None, s_size], dtype=tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer=None,\
                                     activation_fn = tf.nn.relu)
        self.output= slim.fully_connected(hidden, a_size, biases_initializer = None,\
                                         activation_fn = tf.nn.softmax)
        self.chosen_action = tf.argmax(self.output, 1)
        #action을 출력
        
        self.reward_holder = tf.placeholder(shape=[None], dtype = tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype = tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * \
                                tf.shape(self.output)[1] + \
                               self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]),\
                                            self.indexes)
        
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) *
                                   self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx, var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name = str(idx) + '_holder')
            self.gradient_holders.append(placeholder)
            
        self.gradients = tf.gradients(self.loss, tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

In [17]:
tf.reset_default_graph() #Clear the Tensorflow graph.

#configs 

myAgent = Agent(lr=1e-2,s_size=4,a_size=2,h_size=8) #Load the agent.
total_episodes = 5000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 5

In [18]:
init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length = []
        
    gradBuffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
        
    while i < total_episodes:
        s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            #Probabilistically pick an action given our network outputs.
            a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
            a = np.random.choice(a_dist[0],p=a_dist[0])
            a = np.argmax(a_dist == a)

            s1,r,d,_ = env.step(a) #Get our reward for taking an action given a bandit.
            ep_history.append([s,a,r,s1])
            s = s1
            running_reward += r
            if d == True:
                #Update the network.
                ep_history = np.array(ep_history)
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                        myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                for idx,grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                
                total_reward.append(running_reward)
                total_length.append(j)
                break

        
            #Update our running tally of scores.
        if i % 100 == 0:
            print(np.mean(total_reward[-100:]))
        i += 1


C:\Users\qhrrl\AppData\Local\Temp/ipykernel_28528/1959779212.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ep_history = np.array(ep_history)


16.0
27.93
34.85
42.37
58.54
73.0
117.74
156.11
171.39
169.19
162.1
157.3
150.43
157.65
172.32
144.1
141.53
140.42
128.95
148.68
182.25
180.67
182.85
186.91
183.41
191.13
176.87
185.51
180.26
186.26
168.03
176.06
184.66
192.02
196.5
198.18
199.23
198.51
197.02
199.76
198.31
198.44
194.84
191.4
198.05
198.74
196.61
195.26
197.13
197.46
